In [1]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!', '.',',']
stop_words = list(stopwords.words('english'))
data_file = open('intents.json').read()
intents = json.loads(data_file)

Using TensorFlow backend.


In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        #filter out stop words
        words.extend(word for word in w if word not in stop_words)
            # adding documents
        documents.append((w, intent['tag']))
        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

42 documents
18 classes ['breast_cancer', 'calories', 'childhood_earaches', 'cyberbullying', 'day_care', 'eating_plan', 'goodbye', 'greeting', 'hearing_aids', 'kidney_stones', 'options', 'shutup', 'stroke', 'thanks', 'type2_diabetes', 'vegan_diet', 'vegetarian', 'yogurt']
99 unique lemmatized words ["'s", '2', 'advice', 'aid', 'allergy', 'anyone', 'asthma', 'awesome', 'breast', 'bye', 'calorie', 'can', 'cancer', 'care', 'cause', 'chatting', 'child', 'childhood', 'could', 'cyberbullying', 'day', 'developing', 'diabetes', 'diet', 'do', 'doe', 'drinking', 'earache', 'eat', 'eating', 'effect', 'factor', 'genetic', 'get', 'go', 'good', 'goodbye', 'greek', 'hard', 'healthier', 'hearing', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'inherited', 'is', 'kidney', 'later', 'lemon', 'life', 'liquor', 'loss', 'lupus', 'making', 'motivated', 'need', 'next', 'nice', 'offered', 'often', 'patient', 'plan', 'preschool', 'prevent', 'protein', 'provide', 'recommend', 'regular',

In [4]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [5]:
documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Shutup'], 'shutup'),
 (['get', 'out'], 'shutup'),
 (['You', 'are', 'not', 'making', 'sense'], 'shutup'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['How', 'you', 'could', 'help', 'me', '?'], 'options'),
 (['What', 'you', 'can', 'do', '?'], 'options'),
 (['What', 'help', 'you', 'provide', '?'], 'options'),
 (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'),
 (['What', 'support', 'is', 'offered'], 'options'),
 (['Tell', 'me', '

In [6]:
for doc in documents:
    print(doc[0])

['Hi', 'there']
['How', 'are', 'you']
['Is', 'anyone', 'there', '?']
['Hey']
['Hola']
['Hello']
['Good', 'day']
['Bye']
['See', 'you', 'later']
['Goodbye']
['Nice', 'chatting', 'to', 'you', ',', 'bye']
['Till', 'next', 'time']
['Shutup']
['get', 'out']
['You', 'are', 'not', 'making', 'sense']
['Thanks']
['Thank', 'you']
['That', "'s", 'helpful']
['Awesome', ',', 'thanks']
['Thanks', 'for', 'helping', 'me']
['How', 'you', 'could', 'help', 'me', '?']
['What', 'you', 'can', 'do', '?']
['What', 'help', 'you', 'provide', '?']
['How', 'you', 'can', 'be', 'helpful', '?']
['What', 'support', 'is', 'offered']
['Tell', 'me', 'something', 'about', 'type', '2', 'diabetes']
['Tell', 'me', 'something', 'about', 'calories', 'from', 'hard', 'liquor']
['Can', 'a', 'vegan', 'diet', 'cause', 'lupus', 'patient', 'to', 'go', 'into', 'remission', '?']
['Is', 'Greek', 'yogurt', 'healthier', 'than', 'regular', 'yogurt', '?']
['Greek', 'yogurt', 'and', 'regular', 'yogurt', '?']
['I', 'am', 'trying', 'to', 'eat

In [7]:
classes

['breast_cancer',
 'calories',
 'childhood_earaches',
 'cyberbullying',
 'day_care',
 'eating_plan',
 'goodbye',
 'greeting',
 'hearing_aids',
 'kidney_stones',
 'options',
 'shutup',
 'stroke',
 'thanks',
 'type2_diabetes',
 'vegan_diet',
 'vegetarian',
 'yogurt']

In [8]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # record the corrseponding class for each pattern
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [9]:
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Training data created
Epoch 1/200
42/42 [==============================] - 0s 9ms/step - loss: 2.9352 - accuracy: 0.0238
Epoch 2/200
42/42 [==============================] - 0s 238us/step - loss: 2.7831 - accuracy: 0.1429
Epoch 3/200
42/42 [==============================] - 0s 252us/step - loss: 2.7737 - accuracy: 0.1667
Epoch 4/200
42/42 [==============================] - 0s 296us/step - loss: 2.6376 - accuracy: 0.3095
Epoch 5/200
42/42 [==============================] - 0s 324us/step - loss: 2.5456 - accuracy: 0.2857
Epoch 6/200
42/42 [==============================] - 0s 289us/step - loss: 2.4927 - accuracy: 0.3333
Epoch 7/200
42/42 [==============================] - 0s 274us/step - loss: 2.4171 - accuracy: 0.2143
Epoch 8/200
42/42 [==============================] - 0s 238us/step - loss: 2.2841 - accuracy: 0.3810
Epoch 9/200
42/42 [==============================] - 0s 251us/step - loss: 2.3092 - accuracy: 0.2619
Epoch 10/200
42/42 [==============================] - 0s 259us/step - l

42/42 [==============================] - 0s 214us/step - loss: 0.3880 - accuracy: 0.9048
Epoch 82/200
42/42 [==============================] - 0s 201us/step - loss: 0.1419 - accuracy: 0.9762
Epoch 83/200
42/42 [==============================] - 0s 227us/step - loss: 0.1669 - accuracy: 0.9524
Epoch 84/200
42/42 [==============================] - 0s 249us/step - loss: 0.1329 - accuracy: 0.9762
Epoch 85/200
42/42 [==============================] - 0s 231us/step - loss: 0.1390 - accuracy: 0.9762
Epoch 86/200
42/42 [==============================] - 0s 226us/step - loss: 0.1085 - accuracy: 1.0000
Epoch 87/200
42/42 [==============================] - 0s 222us/step - loss: 0.1098 - accuracy: 0.9762
Epoch 88/200
42/42 [==============================] - 0s 188us/step - loss: 0.1041 - accuracy: 0.9762
Epoch 89/200
42/42 [==============================] - 0s 215us/step - loss: 0.0710 - accuracy: 1.0000
Epoch 90/200
42/42 [==============================] - 0s 221us/step - loss: 0.2905 - accuracy: 

42/42 [==============================] - 0s 245us/step - loss: 0.1118 - accuracy: 0.9762
Epoch 161/200
42/42 [==============================] - 0s 268us/step - loss: 0.0484 - accuracy: 1.0000
Epoch 162/200
42/42 [==============================] - 0s 264us/step - loss: 0.0513 - accuracy: 1.0000
Epoch 163/200
42/42 [==============================] - 0s 242us/step - loss: 0.0375 - accuracy: 1.0000
Epoch 164/200
42/42 [==============================] - 0s 262us/step - loss: 0.0364 - accuracy: 1.0000
Epoch 165/200
42/42 [==============================] - 0s 236us/step - loss: 0.0516 - accuracy: 1.0000
Epoch 166/200
42/42 [==============================] - 0s 302us/step - loss: 0.0266 - accuracy: 1.0000
Epoch 167/200
42/42 [==============================] - 0s 303us/step - loss: 0.2078 - accuracy: 0.9286
Epoch 168/200
42/42 [==============================] - 0s 267us/step - loss: 0.0549 - accuracy: 0.9762
Epoch 169/200
42/42 [==============================] - 0s 276us/step - loss: 0.0821 - a

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [11]:
words

["'s",
 '2',
 'advice',
 'aid',
 'allergy',
 'anyone',
 'asthma',
 'awesome',
 'breast',
 'bye',
 'calorie',
 'can',
 'cancer',
 'care',
 'cause',
 'chatting',
 'child',
 'childhood',
 'could',
 'cyberbullying',
 'day',
 'developing',
 'diabetes',
 'diet',
 'do',
 'doe',
 'drinking',
 'earache',
 'eat',
 'eating',
 'effect',
 'factor',
 'genetic',
 'get',
 'go',
 'good',
 'goodbye',
 'greek',
 'hard',
 'healthier',
 'hearing',
 'hello',
 'help',
 'helpful',
 'helping',
 'hey',
 'hi',
 'hola',
 'how',
 'i',
 'inherited',
 'is',
 'kidney',
 'later',
 'lemon',
 'life',
 'liquor',
 'loss',
 'lupus',
 'making',
 'motivated',
 'need',
 'next',
 'nice',
 'offered',
 'often',
 'patient',
 'plan',
 'preschool',
 'prevent',
 'protein',
 'provide',
 'recommend',
 'regular',
 'remission',
 'replace',
 'see',
 'sense',
 'shutup',
 'something',
 'stay',
 'stone',
 'stroke',
 'support',
 'tell',
 'thank',
 'thanks',
 'that',
 'till',
 'time',
 'tool',
 'trying',
 'type',
 'vegan',
 'vegetarian',
 'wa

In [12]:
classes

['breast_cancer',
 'calories',
 'childhood_earaches',
 'cyberbullying',
 'day_care',
 'eating_plan',
 'goodbye',
 'greeting',
 'hearing_aids',
 'kidney_stones',
 'options',
 'shutup',
 'stroke',
 'thanks',
 'type2_diabetes',
 'vegan_diet',
 'vegetarian',
 'yogurt']

In [13]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [14]:
p = bow('How to love Eva Wang', words, show_details = False)

In [15]:
p

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
np.array([p])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [34]:
res = model.predict(np.array([p]))[0]

In [35]:
res

array([1.9567994e-04, 9.8773882e-05, 5.2265310e-05, 3.1873654e-04,
       7.9671678e-05, 1.3645980e-05, 2.8414867e-04, 9.9508154e-01,
       1.1016459e-03, 4.8743255e-05, 3.9591364e-04, 8.4698841e-04,
       7.1388495e-05, 5.0170198e-05, 1.8295288e-05, 3.8299466e-05,
       1.1244571e-03, 1.7953161e-04], dtype=float32)

In [44]:
ERROR_THRESHOLD = 0.25
results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
results.sort(key=lambda x: x[1], reverse=True)


In [45]:
results

[[7, 0.99508154]]

In [41]:
return_list = []
for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

In [46]:
return_list


[{'intent': 'greeting', 'probability': '0.99508154'}]